In [1]:
import torch
import torch.nn as nn
import os
os.chdir('LIA')
from networks.generator import Generator
import argparse
import numpy as np
import torchvision
import os
from PIL import Image
from pathlib import Path
from tqdm import tqdm

/home/lairale/miniconda3/envs/fm/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from run_demo import Demo

In [3]:
arg_dict = dict(model='vox', source_path='./data/vox/test.png', driving_path='./data/vox/driving2.mp4',
               size=256, channel_multiplier=1, latent_dim_style=512, latent_dim_motion=20, save_folder='./res')
class Arg():
    def __init__(self, arg_dict):
        for k, v in arg_dict.items():
            setattr(self, k, v)
args = Arg(arg_dict)

In [4]:
demo = Demo(args)

==> loading model
==> loading data


In [9]:
demo.run()

==> running


100%|█████████████████████████████████████████████████████████████████████████████████| 221/221 [00:11<00:00, 19.29it/s]


In [10]:
print('==> running')
with torch.no_grad():

    vid_target_recon = []

    h_start = demo.gen.enc.enc_motion(demo.vid_target[:, 0, :, :, :])

    for i in tqdm(range(demo.vid_target.size(1))):
        img_target = demo.vid_target[:, i, :, :, :]
        img_recon = demo.gen(demo.img_source, img_target, h_start)
        vid_target_recon.append(img_recon.unsqueeze(2))

    vid_target_recon = torch.cat(vid_target_recon, dim=2)

==> running


100%|█████████████████████████████████████████████████████████████████████████████████| 221/221 [00:11<00:00, 20.00it/s]


In [11]:
vid = vid_target_recon.permute(0, 2, 3, 4, 1)
vid = vid.clamp(-1, 1).cpu()
vid = ((vid - vid.min()) / (vid.max() - vid.min()) * 255).type('torch.ByteTensor')

In [13]:
torchvision.io.write_video(demo.save_path, vid[0], fps=demo.fps)